In [1]:
#Импортируем библиотеки
import pandas as pd 
import datetime as dt
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import json
import csv
from io import StringIO

In [2]:
#Формируем необходимые нам ссылки и заголовки для подключения к yahoo finance
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url_multiples = ('https://finance.yahoo.com/quote/{}/key-statistics?p={}')

#Фомируем источник с полным перечнем тикеров
url_SPX = ('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
spx = pd.read_html(url_SPX)
spx = spx[0]
spx['Symbol'] = spx['Symbol'].str.replace('.','-',regex = True)
ticker_lst_US = list(spx['Symbol'])

#Созадём список тикеров российского рынка
url_MOEX = ('C:/Users/Asus/OneDrive/Рабочий стол/Stocks_Project/Russian_security_list.xlsx')
tickers_RU = pd.read_excel(url_MOEX)
ticker_lst_RU = list(tickers_RU['Ticker'])


In [3]:
#Для выгрузки данных по рынку США
#Формируем основные переменные со списками для всех итераций выгрузки данных
final_df_lst_US = []
final_df_US = pd.DataFrame(final_df_lst_US)


#Прописываем функцию для выгрузки данных
for t in ticker_lst_US:
    try:
        #Созадём список тикеров
        stock = t
        
        #Объявляем данные для выгрузки
        headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        url_multiples = ('https://finance.yahoo.com/quote/{}/key-statistics?p={}')
        
            #Создаём подключение к коду страницы
        respose = requests.get(url_multiples.format(stock,stock), headers = headers)
        soup = BeautifulSoup(respose.text,'lxml')

            #Получаем данные по мультипликаторам, извлекая через метод find_all список по заданному классу
        multiples = soup.find('td',{'class':'Fw(500) Ta(end) Pstart(10px) Miw(60px)'}).text
        multiples = soup.find_all('td',{'class':'Fw(500) Ta(end) Pstart(10px) Miw(60px)'})


            #Присваиваем переменным нужные нам элементы таблицы в yahoo finance с помощью метода элемента списка
            #Мультипликаторы
        Trailing_PE = multiples[2].text
        Frwd_PE = multiples[3].text
        PEG = multiples[4].text
        PS = multiples[5].text
        PB = multiples[6].text
        EV_rev = multiples[7].text
        EV_EBITDA = multiples[8].text
        Beta = multiples[9].text

            #Фундаментальные показатели и коэффициенты
        Profit_mrgn = multiples[40].text
        Oper_mrgn = multiples[41].text
        ROA = multiples[42].text
        ROE = multiples[43].text
        Revenue = multiples[44].text
        EBITDA = multiples[44].text
        Gross_profit = multiples[47].text
        EBITDA = multiples[48].text
        Net_income = multiples[49].text
        EPS_diluted = multiples[50].text


            #Создаём рабочие списки для дальнейшего преобразования данных. При появлении новых показателй их также надо добавить в списки
        multiples_list = [Trailing_PE, Frwd_PE, PEG, PS, PB, EV_rev, EV_EBITDA, Beta, EPS_diluted]
        perce_list = [Profit_mrgn, Oper_mrgn, ROA, ROE]
        fund_list = [Revenue, Gross_profit, EBITDA, Net_income]


            #Создаём функции для обработки полученных с сайта данных

            #функция преобразования данных в числовой тип из мультипликаторов
        float_multiples_list = []

        for i in multiples_list:
            if i=='N/A':
                float_multiples_list.append(i)
            elif 'M' in multiples_list:
                i = i[:-1]
                float_multiples_list.append(float(i))
            else:
                float_multiples_list.append(i)


            #функция преобразования данных в процентный тип
        percentage_list = []
        for i in perce_list:
            if i=='N/A':
                percentage_list.append(i)
            elif ',' in i:
                percentage_list.append(i)
            else:
                i= i[:-1]
                i = float(i)/100
                i = round(i,4)
                percentage_list.append(i)


            #функция преобразования данных в числовой тип из фундаментальных показателей
        fundamentals_list = []
        for i in fund_list:
            if 'T' in i:
                i = i[:-1]
                i = float(i)
                i =  i*1000000000
                fundamentals_list.append(i)
            elif 'B' in i:
                i = i[:-1]
                i = float(i)
                i =  i*1000000
                fundamentals_list.append(i)
            elif 'M' in i:
                i = i[:-1]
                i = float(i)
                i =  i*1000
                fundamentals_list.append(i)
            else:
                fundamentals_list.append(i)



            #Создаём датафреймы из обработанных данных и соединяем их в единый датафрейм
        fundamentals_df = pd.DataFrame({f'{stock}': fundamentals_list}, index=["Revenue","Gross Profit","EBITDA","Net Income"])
        percentage_df = pd.DataFrame({f'{stock}': percentage_list}, index = ["Profit_mrgn", "Oper_mrgn", "ROA", "ROE"])
        multiples_df = pd.DataFrame({f'{stock}': float_multiples_list}, index = ["Trailing_PE", "Frwd_PE", "PEG", "PS", "PB", "EV_rev", "EV_EBITDA", "Beta", "EPS_diluted"])
        combined_df = pd.concat([multiples_df,percentage_df], axis = 0 )
        combined_df = pd.concat([combined_df,fundamentals_df], axis = 0 )
        final_df_US = pd.concat([final_df_US, combined_df], axis = 1)
    except:
        empty_list = []
        empty_list = pd.DataFrame(empty_list)
        final_df_US = pd.concat([final_df_US,empty_list], axis = 0)
        

final_df_US = final_df_US.T
final_df_US = final_df_US.reset_index()
final_df_US


,index,Trailing_PE,Frwd_PE,PEG,PS,PB,EV_rev,EV_EBITDA,Beta,EPS_diluted,Profit_mrgn,Oper_mrgn,ROA,ROE,Revenue,Gross Profit,EBITDA,Net Income
0,MMM,10.57,12.03,6.37,1.72,3.69,2.07,8.36,0.95,9.68,0.1631,0.1496,0.0674,0.3603,33430000.0,15000000.0,6840000.0,5450000.0
1,AOS,46.80,22.42,2.24,3.07,6.25,2.96,33.93,1.29,1.59,0.0649,0.1762,0.1218,0.1349,3740000.0,1330000.0,735400.0,242800.0
2,ABT,32.81,24.51,22.30,4.57,5.07,4.70,18.02,0.66,3.29,0.1398,0.18,0.0632,0.1593,41510000.0,24580000.0,10720000.0,5780000.0
3,ABBV,32.00,12.36,N/A,4.26,18.08,5.20,14.92,0.54,4.33,0.1337,0.3698,0.0944,0.5128,56740000.0,41530000.0,29520000.0,7540000.0
4,ACN,28.14,25.32,2.94,3.17,7.86,3.05,16.64,1.23,11.25,0.1127,0.1535,0.1267,0.3032,63550000.0,19700000.0,10840000.0,7160000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,31.82,26.04,1.97,5.62,N/A,7.11,21.35,1.02,4.26,0.1767,0.3179,0.2384,N/A,6940000.0,3310000.0,2340000.0,1230000.0
499,ZBRA,39.30,19.08,2.56,2.80,5.50,3.15,15.73,1.68,7.64,0.0709,0.166,0.0878,0.1408,5750000.0,2620000.0,1150000.0,408000.0
500,ZBH,66.53,19.92,3.39,4.21,2.45,4.97,20.96,1.02,2.14,0.0633,0.1986,0.0412,0.0373,7110000.0,4920000.0,2340000.0,449700.0
501,ZION,5.25,6.25,0.71,1.39,0.96,N/A,N/A,1.08,5.85,0.2924,0.3958,0.0101,0.1582,3100000.0,3030000.0,N/A,874000.0


In [4]:
#Заполняем нулями все N/A значения
data = final_df_US.copy()
data = data.replace("N/A",0)

#Выбираем метрику с наименьшим к-вом отсуствующих значений для большей точности оценки
data[data!=0].count()

#Выбрали показатели P/S, Frwd_PE, EV_rev, Oper_mrgn, Profit_mrgn

index           503
Trailing_PE     480
Frwd_PE         496
PEG             415
PS              503
PB              474
EV_rev          477
EV_EBITDA       454
Beta            494
EPS_diluted     501
Profit_mrgn     496
Oper_mrgn       496
ROA             494
ROE             466
Revenue         496
Gross Profit    490
EBITDA          470
Net Income      496
dtype: int64

In [ ]:
#Добавляем информацию по секторам
data = data.rename(columns = {'index':'Ticker'})
data_cut = data[['Ticker', 'Frwd_PE', 'EV_rev', 'Oper_mrgn', 'Profit_mrgn']]

#Забираем с Yahoo Finance данные о секторах
data_tickers = list(data['Ticker'])

sector_info = dict()

for ticker in data_tickers: 
    try:  
        url = f'https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}'
        respose = requests.get(url.format(ticker,ticker), headers = headers)
        soup = BeautifulSoup(respose.text,'lxml')
        profile = soup.find_all('span',{'class':'Fw(600)'})
        sector = profile[1].text
        sector_info[ticker] = [sector]
    except:
        pass
sector_info = pd.DataFrame.from_dict(sector_info, orient='index')
sector_info = sector_info.rename(columns = {0:'Sector'})
sector_info = sector_info.reset_index()
sector_info = sector_info.rename(columns = {'index':'Ticker'})
sector_info.head(15)

In [ ]:
data_cut

In [ ]:
#Считаем медианные значения по выбранным раннее параметрам в разрезе секторов и отраслей, а также медианные по всему рынку
data_cut_1 = data_cut.copy()
data_cut_1 = data_cut_1.merge(sector_info, how = 'left', left_on = 'Ticker', right_on = 'Ticker')
data_cut_1[['Frwd_PE','EV_rev']] = data_cut_1[['Frwd_PE','EV_rev']].apply(lambda x : pd.to_numeric(x, errors='coerce'))
data_cut_1 = data_cut_1.dropna()
data_cut_groupped = data_cut_1.groupby('Sector').median().reset_index()
names = ['Sector','Sec_Frwd_PE','Sec_EV_rev','Sec_Oper_mrgn','Sec_Profit_mrgn']
data_cut_groupped.columns = names

data_cut_1 = data_cut_1.merge(data_cut_groupped, how = 'left', left_on = 'Sector', right_on = 'Sector')

#Смотрим, по каким бумагам наблюдается недооценка по более чем 3 фундаментальным показателям
F_PE_underA = list(data_cut_1['Ticker'][(data_cut_1['Frwd_PE']-data_cut_1['Sec_Frwd_PE'])<0])
EV_rev_underA = list(data_cut_1['Ticker'][(data_cut_1['EV_rev']-data_cut_1['Sec_EV_rev'])<0])
Oper_m_underA = list(data_cut_1['Ticker'][(data_cut_1['Oper_mrgn']-data_cut_1['Sec_Oper_mrgn'])<0])
Profit_m_underA = list(data_cut_1['Ticker'][(data_cut_1['Profit_mrgn']-data_cut_1['Sec_Profit_mrgn'])<0])

under_A_stocks = pd.DataFrame(F_PE_underA+EV_rev_underA+Oper_m_underA+Profit_m_underA, columns=['Ticker'])
under_A_stocks = pd.DataFrame(under_A_stocks.value_counts(), columns=['cnt'])
under_A_stocks = under_A_stocks[under_A_stocks['cnt']>=3].reset_index()

#Получаем список потенциально недооценённых акций
under_A_stocks = list(under_A_stocks['Ticker'])
under_A_stocks